# Классификация надежности паролей, с использованием Машинного Обучения

## Загрузка имеющегося набора данных. Исследование и корректировка данных в наборе

In [433]:
import pandas as pd
import numpy as np
import numpy.random

In [434]:
filepath = "data.csv"
data = pd.read_csv(filepath, on_bad_lines='skip')

In [435]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 669640 entries, 0 to 669639
Data columns (total 2 columns):
 #   Column    Non-Null Count   Dtype 
---  ------    --------------   ----- 
 0   password  669639 non-null  object
 1   strength  669640 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 10.2+ MB


In [436]:
data.head()

,password,strength
0,kzde5577,1
1,kino3434,1
2,visi7k1yr,1
3,megzy123,1
4,lamborghin1,1


Видим, что наш набор данных состоит из паролей и их сложностей. Посмотрим, на какие уровни сложности разбиваются имеющиеся пароли:

In [437]:
data['strength'].value_counts()

strength
1    496801
0     89702
2     83137
Name: count, dtype: int64

Пароли разбиты на три уровня сложности: **weak**(слабый) - 0, **medium**(средний) - 1 и **strong**(сильный) - 2. Проверим, чтобы в таблице не было пропусков,которые могут помешать в работе. Если они присутствуют - удалим.

In [438]:
data[data['password'].isnull()]

,password,strength
367579,NaN,0


In [439]:
data.dropna(inplace=True)
data[data['password'].isnull()]
type(data)

pandas.core.frame.DataFrame

### Немного о исследуемом наборе данных
Рассмотрим используемый датасет подробнее. Он состоит из паролей, взятых из утечки 000webhost. Как же устанавливалась сложность паролей? Все довольно просто. Для оценки надежности были использованы алгоритмы надежности от Twitter, Microsoft и battle. Те пароли, которые получили одинаковую оценку от каждого из сервисов, были добавлены в используемый набор данных. Перейдем к анализу.

## Предобработка данных
Перемешаем имеющиеся данные

In [440]:
passwords = np.array(data)
np.random.shuffle(passwords)

Разделим входы и указания учителя

In [441]:
X = passwords[:, 0]
y = passwords[:, 1].astype(np.double)

Так как большая часть моделей машинного обучения, в том числе и логистическая регрессия, используемая нами, работают с **численными данными**, то нам необходимо преобразовать пароли в вектора чисел. Для этого, мы будем использовать **TfidfVectorizer** (Term Frequency-Inverse Document Frequency Vectorizer) - это инструмент для преобразования коллекции текстовых документов в матрицу TF-IDF. 
Так как, по умолчанию, TfidfVectorizer работает с текстами, и разбивает их на отдельные слова (каждый отдельный токен - слово), нам необходимо указать ему другой способ разбивки на токены, т.к. работаем не с текстами, а со словами. Сделаем простейший токенизатор, который будет разбивать каждый пароль на отдельные символы(токены)

In [442]:
def getTokens(inputString):
	tokens = []
	for i in inputString:
		tokens.append(i)
	return tokens

Далее, проводим описанное ранее преобразование наших данных, а также разбиваем данные на тестовые и тренировочные.

In [443]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
    
vectorizer = TfidfVectorizer(tokenizer=getTokens)
X = vectorizer.fit_transform(X)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=42)

/Users/Work/anaconda3/lib/python3.11/site-packages/sklearn/feature_extraction/text.py:525: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


## Обучение модели и получение результатов
Обучаем модель логистической регрессии, после чего, выводим точность ее работы на тестовых данных

In [444]:
from sklearn import linear_model
lgs = linear_model.LogisticRegression(penalty='l2',multi_class='ovr')
lgs.fit(X_train, y_train)
print(lgs.score(X_test, y_test))

0.8113762618720507


Для примера, придумаем пароли и предскажем их точность:

In [445]:
X_predict = ['123456789','qwerty','qwerty1','1qwertq1','1qwert1qwer','1qwert1qwert','1qwerty#','!21q$w)er(ty#%','111111111111111111']
X_predict = vectorizer.transform(X_predict)
y_Predict = lgs.predict(X_predict)
print(y_Predict)

[1. 0. 1. 0. 0. 1. 2. 2. 0.]


## А можно ли лучше?
Полученная точность не является идеальной. Попробуем увеличить точность нашей модели. Для этого, добавим признаков, на которые она может ориентироваться. Например, добавим длину пароля в вектор признаков.

In [472]:
def getTokens2(inputString):
    tokens = []
    for i in inputString:
        tokens.append(i)
    tokens.append(str(len(inputString)))
    return tokens

Еще раз выделим данные

In [473]:
X2 = passwords[:, 0]
y2 = passwords[:, 1].astype(np.double)

In [474]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
    
vectorizer = TfidfVectorizer(tokenizer=getTokens2)
X2 = vectorizer.fit_transform(X2)

X2_train, X2_test, y2_train, y2_test = train_test_split(X2, y2, test_size=0.20, random_state=42)

/Users/Work/anaconda3/lib/python3.11/site-packages/sklearn/feature_extraction/text.py:525: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


In [475]:
from sklearn import linear_model
lgs2 = linear_model.LogisticRegression(penalty='l2',multi_class='ovr')
lgs2.fit(X2_train, y2_train)
print(lgs2.score(X2_test, y2_test))

0.9590974254823488


Продемонстрируем работу нашей модели на практике:

In [476]:
X2_predict = ['123456789','qwerty','qwerty1','1qwertq1','1qwert1qwer','1qwert1qwert','1qwerty#','!21q$w)er(ty#%','111111111111111111']
X2_predict = vectorizer.transform(X2_predict)
y2_Predict = lgs2.predict(X2_predict)
print(y2_Predict)

[1. 0. 0. 1. 1. 1. 1. 2. 0.]
